# Examen IB: Conversión un número decimal a su interpretación en IEEE 754 de 32 bits en Python

## David Alejandro Puga Novoa

### GR2CC

Realizar un programa (Python/Java) que permita representar en formato IEEE de 32 bits de coma flotante un número decimal. 

Se requiere:

1. Código Fuente del programa (*.py, *.ipynb, *.java)

2. Presentación Jupyter-Rise o Org-Emacs que indique las partes fundamentales del código y su ejecución.

3. Presentación oral.

4. El programa ha de imprimir:

- a. el número dado

- b. el bit del signo

- c. el exponente

- d. el exponente sesgado

- e. la mantisa

- f. el equivalente binario en formato IEEE de 32 bits

- g. el equivalente hexadecimal (puede usar una librería para este punto)

Acontinuación, se mostrará como se realizo el código y la explicación de cada función con un ejemplo.

## Función decimal_a_binario

Convierte un número decimal en su representación binaria, sin incluir el signo. Retorna el signo y la representación binaria.

In [5]:
def decimal_a_binario(decimal):
    if decimal == 0:
        return '0'
    signo = '0' if decimal > 0 else '1'
    decimal = abs(decimal)
    parte_entera = int(decimal)
    parte_fraccionaria = decimal - parte_entera

    # Convertir parte entera a binario
    binario_entero = ''
    if parte_entera == 0:
        binario_entero = '0'
    while parte_entera > 0:
        binario_entero = str(parte_entera % 2) + binario_entero
        parte_entera = parte_entera // 2

    # Convertir parte fraccionaria a binario
    binario_fraccionario = ''
    while parte_fraccionaria > 0:
        parte_fraccionaria *= 2
        if parte_fraccionaria >= 1:
            binario_fraccionario += '1'
            parte_fraccionaria -= 1
        else:
            binario_fraccionario += '0'
        if len(binario_fraccionario) > 23:
            break

    binario = binario_entero + '.' + binario_fraccionario
    return signo, binario

### Explicación
1. **Verificar si el número es cero**: Si el decimal es 0, la función retorna '0'.
2. **Determinar el signo**: Si el número es positivo, el signo es '0'. Si es negativo, el signo es '1'.
3. **Convertir la parte entera a binario**: Se convierte la parte entera del número decimal a su representación binaria dividiendo sucesivamente por 2 y acumulando el resto.
4. **Convertir la parte fraccionaria a binario**: Se multiplica la parte fraccionaria por 2 y se acumulan los dígitos binarios resultantes hasta obtener 23 bits, porque ese es el tamaño de la mantisa para 32 bits.
5. **Combinar parte entera y fraccionaria**: Se combinan ambas partes separadas por un punto decimal ('.').

### Ejemplo
- Ejemplo para el número -10.75.

In [2]:
decimal_a_binario(-10.75)

('1', '1010.11')

## Función normalizar_binario

Normaliza la representación binaria que se obtuvo, calculando el exponente y la mantisa.

In [6]:
def normalizar_binario(binario):
    if '.' not in binario:
        binario += '.0'
    
    parte_entera, parte_fraccionaria = binario.split('.')
    if '1' in parte_entera:
        exponente = len(parte_entera) - 1
        mantisa = parte_entera[1:] + parte_fraccionaria
    else:
        exponente = -parte_fraccionaria.find('1') - 1
        mantisa = parte_fraccionaria[parte_fraccionaria.find('1') + 1:]

    return exponente, mantisa

### Explicación
1. **Asegurarse de que hay un punto decimal**: Si no hay un punto decimal en la representación binaria, se añade '.0'.
2. **Separar la parte entera y la fraccionaria**: Se divide la cadena binaria en parte entera y parte fraccionaria usando el punto decimal como separador.
3. **Calcular el exponente y la mantisa**: 
   - Si la parte entera contiene un '1', el exponente es la longitud de la parte entera menos uno, y la mantisa es la parte entera (sin el primer dígito) concatenada con la parte fraccionaria.
   - Si la parte entera no contiene un '1', se busca el primer '1' en la parte fraccionaria. El exponente es el negativo de la posición de ese '1' menos uno, y la mantisa es la parte fraccionaria después del primer '1'.

### Ejemplo

- Continuemos con el ejemplo del número -10.75. Donde su binario sin signo es 1010.11. Entonces procedemos a normalizarlo.

In [4]:
normalizar_binario('1010.11')

(3, '01011')

## Función obtener_exponente_y_mantisa

Obtiene el exponente normalizado, el sesgo del exponente y ajusta la mantisa a 23 bits.

In [7]:
def obtener_exponente_y_mantisa(binario):
    exponente, mantisa = normalizar_binario(binario)
    sesgo_exponente = exponente + 127
    mantisa = mantisa[:23].ljust(23, '0') 
    return exponente, sesgo_exponente, mantisa

### Explicación
1. **Normalizar el binario**: Se llama a la función normalizar_binario para obtener el exponente y la mantisa.
2. **Calcular el exponente con sesgo**: El sesgo para el exponente en IEEE 754 de 32 bits es 127. Para obtenerlo se suma 127 al exponente normalizado.
3. **Ajustar la mantisa**: Se ajusta la mantisa a 23 bits, rellenando con ceros a la derecha si es necesario.

### Ejemplo
- Siguiendo con el ejemplo del -10.75 donde su binario (sin signo) fue 1010.11, obtendremos el exponente y la mantisa.

In [6]:
obtener_exponente_y_mantisa('1010.11')

(3, 130, '01011000000000000000000')

## Función decimal_a_ieee754

Combina todas las funciones anteriores para convertir el número decimal a su representación IEEE 754 de 32 bits.

In [8]:
def decimal_a_ieee754(decimal):
    signo, binario = decimal_a_binario(decimal)
    exponente, sesgo_exponente, mantisa = obtener_exponente_y_mantisa(binario)
    
    bit_signo = signo
    bits_exponente = format(sesgo_exponente, '08b')
    bits_mantisa = mantisa
    
    ieee754 = f"{bit_signo}{bits_exponente}{bits_mantisa}"
    return signo, binario, exponente, sesgo_exponente, bits_exponente, bits_mantisa, ieee754

### Explicación
1. **Convertir decimal a binario**: Se llama a la función decimal_a_binario para obtener el signo y la representación binaria del número sin signo.
2. **Obtener el exponente y la mantisa**: Se llama a la función obtener_exponente_y_mantisa para obtener el exponente normalizado, el sesgo del exponente y la mantisa.
3. **Convertir exponente y mantisa a binario**: Se convierte el sesgo del exponente a una cadena binaria de 8 bits.
4. **Ensamblar la representación IEEE 754**: Se combina el signo, el exponente en binario y la mantisa en una sola cadena para formar la representación final en IEEE 754.

### Ejemplo
- Seguimos con el número -10.75 al tener y al tener todo en conjunto convertimos a IEEE 754.

In [8]:
decimal_a_ieee754(-10.75)

('1',
 '1010.11',
 3,
 130,
 '10000010',
 '01011000000000000000000',
 '11000001001011000000000000000000')

## Función Principal main

Ejecuta el main para que se realice el proceso de conversión y muestra los resultados.

In [9]:
def main():
    decimal = float(input("Introduce un número decimal (positivo o negativo): "))
    if decimal == 0:
        print(f"IEE 754 completo: 00000000000000000000000000000000")
    else:
        signo, binario, exponente, sesgo_exponente, bits_exponente, bits_mantisa, ieee754 = decimal_a_ieee754(decimal)
        hexadecimal = hex(int(ieee754, 2))
        print(f"Decimal: {decimal}")
        print(f"Número en binario (sin signo): {binario}")
        print(f"Representación IEEE 754:")
        print(f"  Signo: {signo}")
        print(f"  Exponente (sin sesgo): {exponente}")
        print(f"  Sesgo del exponente (decimal): {sesgo_exponente}")
        print(f"  Sesgo del exponente (binario): {bits_exponente}")
        print(f"  Mantisa: {bits_mantisa}")
        print(f"  IEEE 754 completo: {ieee754}")
        print(f"Equivalente hexadecimal: {hexadecimal}")

if __name__ == "__main__":
    main()

Introduce un número decimal (positivo o negativo):  -10.75


Decimal: -10.75
Número en binario (sin signo): 1010.11
Representación IEEE 754:
  Signo: 1
  Exponente (sin sesgo): 3
  Sesgo del exponente (decimal): 130
  Sesgo del exponente (binario): 10000010
  Mantisa: 01011000000000000000000
  IEEE 754 completo: 11000001001011000000000000000000
Equivalente hexadecimal: 0xc12c0000


 ### Finalmente es como se nos muestra cada uno de los literales que se solicitaron para este ejemplo.